In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression

import RPi.GPIO as GPIO

import smbus
import threading
import time

pin = 18 # PWM pin num 18

I2C_CH = 1
BH1750_DEV_ADDR = 0x23

CONT_H_RES_MODE     = 0x10
CONT_H_RES_MODE2    = 0x11
CONT_L_RES_MODE     = 0x13
ONETIME_H_RES_MODE  = 0x20
ONETIME_H_RES_MODE2 = 0x21
ONETIME_L_RES_MODE  = 0x23

# 모터 셋팅
out1 = 13
out2 = 11
out3 = 15
out4 = 12

i=0
positive=0
negative=0
y=0

GPIO.setmode(GPIO.BOARD)
GPIO.setup(out1,GPIO.OUT)
GPIO.setup(out2,GPIO.OUT)
GPIO.setup(out3,GPIO.OUT)
GPIO.setup(out4,GPIO.OUT)


def move_motor(x):
    if x>0 and x<=400:
        for y in range(x,0,-1):
            if negative==1:
                if i==7:
                    i=0
                else:
                    i=i+1
                y=y+2
                negative=0
            positive=1
            if i==0:
                GPIO.output(out1,GPIO.HIGH)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.LOW)
            elif i==1:
                GPIO.output(out1,GPIO.HIGH)
                GPIO.output(out2,GPIO.HIGH)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.LOW)
                time.sleep(1)
            elif i==2:  
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.HIGH)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.LOW)
            elif i==3:    
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.HIGH)
                GPIO.output(out3,GPIO.HIGH)
                GPIO.output(out4,GPIO.LOW)
            elif i==4:  
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.HIGH)
                GPIO.output(out4,GPIO.LOW)
            elif i==5:
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.HIGH)
                GPIO.output(out4,GPIO.HIGH)
            elif i==6:    
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.HIGH)
            elif i==7:    
                GPIO.output(out1,GPIO.HIGH)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.HIGH)
            if i==7:
                i=0
                continue
            i=i+1
      
      
    elif x<0 and x>=-400:
        x=x*-1
        for y in range(x,0,-1):
            if positive==1:
                if i==0:
                    i=7
                else:
                    i=i-1
                y=y+3
                positive=0
            negative=1
            if i==0:
                GPIO.output(out1,GPIO.HIGH)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.LOW)
            elif i==1:
                GPIO.output(out1,GPIO.HIGH)
                GPIO.output(out2,GPIO.HIGH)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.LOW)
            elif i==2:  
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.HIGH)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.LOW)
            elif i==3:    
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.HIGH)
                GPIO.output(out3,GPIO.HIGH)
                GPIO.output(out4,GPIO.LOW)
            elif i==4:  
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.HIGH)
                GPIO.output(out4,GPIO.LOW)
            elif i==5:
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.HIGH)
                GPIO.output(out4,GPIO.HIGH)
            elif i==6:    
                GPIO.output(out1,GPIO.LOW)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.HIGH)
            elif i==7:    
                GPIO.output(out1,GPIO.HIGH)
                GPIO.output(out2,GPIO.LOW)
                GPIO.output(out3,GPIO.LOW)
                GPIO.output(out4,GPIO.HIGH)
            if i==0:
                i=7
                continue
            i=i-1 


def read_data():
    global df
    # 기본 파일 로드
    df = pd.read_csv("data.csv") 
    
def fit_model():
    read_data()
    X = df.iloc[:,0:-1]
    Y = df["Angle"]

    lm = LinearRegression(fit_intercept=True, normalize=True, n_jobs=None)
    lm.fit(X, Y)
        
    accuracy = lm.score(X, Y)
    print(accuracy)
    
    return lm

def angle(value):
    lm = fit_model()
    
    X = df.iloc[-1 ,0:-1]
    predict = lm.predict([X])
    if (predict - value >= 45 || value - predict >= 45):
        df.iloc[-1, -1] = lm.predict([X])
    else:
        df.iloc[-1, -1] = value


def readIlluminance():
    i2c = smbus.SMBus(I2C_CH)
    luxBytes = i2c.read_i2c_block_data(BH1750_DEV_ADDR, CONT_H_RES_MODE, 2)
    lux = int.from_bytes(luxBytes, byteorder='big')
    i2c.close()
    return lux


def readIlluminanceThread():
    while True:
        print('{0} lux'.format(readIlluminance()))
        list = [readIlluminance(),readIlluminance(),readIlluminance(),readIlluminance(), 0]
        index = list.index(max(list))
        value = 0
        
        if index == 1:
            value = 90
        elif index == 2:
            value = 180
        elif index == 3:
            value = 270
        else:
            value = 0
            
        read_data()
        df.loc[len(df)] = [n for n in list]
        angle(value)
        df.to_csv("data.csv", index = False) # 데이터 인풋
        
        time.sleep(60)
        

def fit_modelThread():
    pre_angle = 0
    move_angle = 0
    while True:
        read_data()
        motor_angle = df.iloc[-1, -1] # 이게 움직여야하는 각도
        
        if motor_angle >= pre_angle:
            move_angle = motor_angle - pre_angle
        else:
            move_angle = 360 - pre_angle + motor_angle
            
        #move_motor(move_angle)
        move_motor(motor_angle)
        
        pre_angle = motor_angle
        
        time.sleep(600) #10분


thd = threading.Thread(target=readIlluminanceThread)
# 쓰레드를 데몬으로 설정
thd.daemon = True
# 쓰레드 시작
thd.start()


learn = threading.Thread(target=fit_modelThread)
# 쓰레드를 데몬으로 설정
learn.daemon = True
# 쓰레드 시작
learn.start()


GPIO.cleanup()

0.9867814359294742
[207.16029722 186.14031047 259.11349461 289.46204442 302.31328507]
